# AskNews

> [AskNews](https://asknews.app) infuses any LLM with the latest global news (or historical news), using a single natural language query. Specifically, AskNews is enriching over 300k articles per day by translating, summarizing, extracting entities, and indexing them into hot and cold vector databases. AskNews puts these vector databases on a low-latency endpoint for you. When you query AskNews, you get back a prompt-optimized string that contains all the most pertinent enrichments (e.g. entities, classifications, translation, summarization). This means that you do not need to manage your own news RAG, and you do not need to worry about how to properly convey news information in a condensed way to your LLM.
> AskNews is also committed to transparency, which is why our coverage is monitored and diversified across hundreds of countries, 13 languages, and 50 thousand sources. If you'd like to track our source coverage, you can visit our [transparency dashboard](https://asknews.app/en/transparency).

## Setup

The integration lives in the `langchain-community` package. We also need to install the `asknews` package itself.

```bash
pip install -U langchain-community asknews
```

We also need to set our AskNews API credentials, which can be generated at the [AskNews console](https://my.asknews.app).

In [1]:
import getpass
import os

os.environ["ASKNEWS_CLIENT_ID"] = getpass.getpass()
os.environ["ASKNEWS_CLIENT_SECRET"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

In [13]:
from langchain_community.retrievers import AskNewsRetriever

retriever = AskNewsRetriever(k=3)

retriever.invoke("impact of fed policy on the tech sector")

[Document(page_content='[1]:\ntitle: US Stock Market Declines Amid High Interest Rates\nsummary: The US stock market has experienced a significant decline in recent days, with the S&P 500 index falling 9.94 points or 0.2% to 4320.06. The decline was attributed to interest rates, which are expected to remain high for a longer period. The yield on 10-year Treasury notes rose to 4.44%, the highest level since 2007, which has a negative impact on stock prices. The high interest rates have also affected the technology sector, with companies such as Intel and Microsoft experiencing declines. The auto sector is also experiencing fluctuations, with General Motors and Ford experiencing declines. The labor market is also facing challenges, with workers demanding higher wages and benefits, which could lead to increased inflation. The Federal Reserve is expected to keep interest rates high for a longer period, which could have a negative impact on the stock market. Some economists expect the Fed t

In [19]:
# you have full control on filtering by category, time, pagination, and even the search method you use.
from datetime import datetime, timedelta

start = (datetime.now() - timedelta(days=7)).timestamp()
end = datetime.now().timestamp()

retriever = AskNewsRetriever(
    k=3,
    categories=["Business", "Technology"],
    start_timestamp=int(start),  # defaults to 48 hours ago
    end_timestamp=int(end),  # defaults to now
    method="kw",  # defaults to "nl", natural language, can also be "kw" for keyword search
    offset=10,  # allows you to paginate results
)

retriever.invoke("federal reserve S&P500")

[Document(page_content="[1]:\ntitle: US Stocks End Week with Modest Gains Ahead of Inflation Data\nsummary: US stocks ended the week with modest gains, marking another weekly advance for the three main indices, as investors evaluated comments from Federal Reserve officials and awaited crucial inflation data next week. The S&P 500 and Dow Jones experienced slight increases, while the Nasdaq closed almost unchanged. The Dow Jones recorded its largest weekly percentage gain since mid-December. Federal Reserve officials' comments generated expectations, with market participants awaiting inflation data to be released next week. The data includes the Consumer Price Index (CPI) and Producer Price Index (PPI) from the Department of Labor, which is expected to provide insight into progress towards the 2% inflation target.\nsource: El Cronista\npublished: May 10 2024 23:35\nLocation: US\nOrganization: Dow Jones, Department of Labor, Nasdaq, Federal Reserve\nDate: next week, mid-December\nclassif

## Chaining

We can easily combine this retriever in to a chain.

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """The following news articles may come in handy for answering the question:

{context}

Question:

{question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-1106-preview")
    | StrOutputParser()
)

In [12]:
chain.invoke({"question": "What is the impact of fed policy on the tech sector?"})

"According to the information provided in the second news article, the impact of Federal Reserve policy on the technology sector has been negative. The article mentions that due to expectations that interest rates will remain high for a longer period, the US stock market has experienced a significant decline. This rise in interest rates has particularly affected the technology sector, with companies such as Intel and Microsoft experiencing declines. High interest rates can lead to increased borrowing costs for businesses, which can dampen investment and spending. In the technology sector, where companies often rely on borrowing to fund research and development and other growth initiatives, higher rates can be especially challenging.\n\nTherefore, the Federal Reserve's policy of maintaining high interest rates has had a detrimental effect on tech stocks, contributing to a decrease in their market valuations."